# mapa_accidentes_severidad_cluster

In [37]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import numpy as np
from pyproj import Proj, transform
from folium import MacroElement
from jinja2 import Template

# Cargar el archivo CSV
file_path = "data/2024_Accidentalidad.csv"
df = pd.read_csv(file_path, encoding='ISO-8859-1', delimiter=';')

# Asegurarse de que las columnas sean cadenas para aplicar reemplazos
df['coordenada_x_utm'] = df['coordenada_x_utm'].astype(str)
df['coordenada_y_utm'] = df['coordenada_y_utm'].astype(str)

# Reemplazar comas (,) por puntos (.) en las coordenadas UTM para poder convertirlas correctamente
df['coordenada_x_utm'] = df['coordenada_x_utm'].str.replace(',', '.')
df['coordenada_y_utm'] = df['coordenada_y_utm'].str.replace(',', '.')

# Convertir las coordenadas a numéricas
df['coordenada_x_utm'] = pd.to_numeric(df['coordenada_x_utm'], errors='coerce')
df['coordenada_y_utm'] = pd.to_numeric(df['coordenada_y_utm'], errors='coerce')

# Filtrar filas con coordenadas válidas
df = df.dropna(subset=['coordenada_x_utm', 'coordenada_y_utm'])

# Definir la proyección UTM 30N (usada en España) y la proyección geográfica
utm_proj = Proj(proj='utm', zone=30, ellps='WGS84', south=False)
geo_proj = Proj(proj='latlong', datum='WGS84')

# Convertir las coordenadas UTM a latitud y longitud
def convertir_utm_a_latlon(x, y):
    try:
        lon, lat = transform(utm_proj, geo_proj, x, y)
        return lat, lon
    except:
        return np.nan, np.nan

# Aplicar la conversión
df[['Latitud', 'Longitud']] = df.apply(lambda row: convertir_utm_a_latlon(row['coordenada_x_utm'], row['coordenada_y_utm']), axis=1, result_type='expand')

# Filtrar filas con coordenadas geográficas válidas
df = df.dropna(subset=['Latitud', 'Longitud'])

# Crear el mapa centrado en Madrid, España
mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=12)

# Añadir la capa de vista satélite usando TileLayer
folium.TileLayer(
    tiles='http://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', 
    attr='Google Satellite', 
    name='Google Satellite'
).add_to(mapa)

# Añadir la capa de mapa base
folium.TileLayer(
    tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 
    attr='Mapa Base', 
    name='Mapa Base'
).add_to(mapa)

# Crear clústeres para los accidentes agrupados por severidad
cluster_fallecidos = MarkerCluster(name='Accidentes con Fallecidos').add_to(mapa)
cluster_heridos = MarkerCluster(name='Accidentes con Heridos').add_to(mapa)
cluster_sin_asistencia = MarkerCluster(name='Accidentes sin Asistencia Sanitaria').add_to(mapa)

# Filtrar accidentes por severidad usando la columna 'lesividad'
accidentes_fallecidos = df[df['lesividad'].str.contains('Fallecido', case=False, na=False)]  # Filtrar por accidentes con fallecidos
accidentes_heridos = df[df['lesividad'].str.contains('Asistencia sanitaria', case=False, na=False)]  # Filtrar por accidentes con heridos
accidentes_sin_asistencia = df[df['lesividad'].str.contains('Sin asistencia sanitaria', case=False, na=False)]  # Filtrar por sin asistencia

# Añadir marcadores para los accidentes con fallecidos al clúster correspondiente
for _, row in accidentes_fallecidos.iterrows():
    folium.Marker(
        location=[row['Latitud'], row['Longitud']],
        popup=(
            f"Fecha: {row['fecha']}<br>"
            f"Hora: {row['hora']}<br>"
            f"Localización: {row['localizacion']}<br>"
            f"Tipo de Accidente: {row['tipo_accidente']}<br>"
            f"Estado Meteorológico: {row['estado_meteorológico']}<br>"
            f"Lesividad: {row['lesividad']}"
        ),
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(cluster_fallecidos)

# Añadir marcadores para los accidentes con heridos al clúster correspondiente
for _, row in accidentes_heridos.iterrows():
    folium.Marker(
        location=[row['Latitud'], row['Longitud']],
        popup=(
            f"Fecha: {row['fecha']}<br>"
            f"Hora: {row['hora']}<br>"
            f"Localización: {row['localizacion']}<br>"
            f"Tipo de Accidente: {row['tipo_accidente']}<br>"
            f"Estado Meteorológico: {row['estado_meteorológico']}<br>"
            f"Lesividad: {row['lesividad']}"
        ),
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(cluster_heridos)

# Añadir marcadores para los accidentes sin asistencia sanitaria al clúster correspondiente
for _, row in accidentes_sin_asistencia.iterrows():
    folium.Marker(
        location=[row['Latitud'], row['Longitud']],
        popup=(
            f"Fecha: {row['fecha']}<br>"
            f"Hora: {row['hora']}<br>"
            f"Localización: {row['localizacion']}<br>"
            f"Tipo de Accidente: {row['tipo_accidente']}<br>"
            f"Estado Meteorológico: {row['estado_meteorológico']}<br>"
            f"Lesividad: {row['lesividad']}"
        ),
        icon=folium.Icon(color='green', icon='info-sign')
    ).add_to(cluster_sin_asistencia)

# Añadir un control de capas para mostrar/ocultar los diferentes grupos (oculto inicialmente)
folium.LayerControl(collapsed=True).add_to(mapa)

# Crear un control personalizado para mostrar/ocultar el control de capas con un botón
class ToggleLayerControl(MacroElement):
    _template = Template(u"""
        <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000;">
            <button onclick="var lc = document.getElementsByClassName('leaflet-control-layers')[0]; 
                             lc.style.display = lc.style.display === 'block' ? 'none' : 'block';">
                Mostrar Opciones
            </button>
        </div>
    """)

    def __init__(self):
        super().__init__()


# Añadir el control personalizado al mapa
mapa.get_root().add_child(ToggleLayerControl())

# Guardar el mapa en un archivo HTML
mapa.save('mapa.html')
print("Mapa guardado como 'mapa.html'. Verifica este archivo en el navegador para ver las nuevas funcionalidades.")



/var/folders/mq/mnz2dh_90hx7l6l083kjws6w0000gn/T/ipykernel_62256/1401545912.py:35: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(utm_proj, geo_proj, x, y)


Mapa guardado como 'mapa_accidentes_severidad_control.html'. Verifica este archivo en el navegador para ver las nuevas funcionalidades.


# mapa_accidentes_cluster

In [28]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import numpy as np

# Asumimos que el DataFrame 'df' ya está creado y tiene las columnas 'Latitud' y 'Longitud'
# y que hemos hecho la conversión correcta de UTM a latitud y longitud.

# Crear un mapa centrado en Madrid, España
mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=12)

# Usar MarkerCluster para agrupar los marcadores y mejorar el rendimiento del mapa
marker_cluster = MarkerCluster().add_to(mapa)

# Agregar los marcadores al clúster
for _, row in df.iterrows():
    if pd.notnull(row['Latitud']) and pd.notnull(row['Longitud']):
        folium.Marker(
            location=[row['Latitud'], row['Longitud']],
            popup=(
                f"Fecha: {row['fecha']}<br>"
                f"Hora: {row['hora']}<br>"
                f"Localización: {row['localizacion']}<br>"
                f"Tipo de Accidente: {row['tipo_accidente']}<br>"
                f"Estado Meteorológico: {row['estado_meteorológico']}<br>"
                f"Lesividad: {row['lesividad']}"
            ),
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(marker_cluster)

# Guardar el mapa con los marcadores agrupados en clúster
mapa.save('mapa_accidentes_cluster.html')
print("Mapa guardado como 'mapa_accidentes_cluster.html'.")

Mapa guardado como 'mapa_accidentes_cluster.html'.
